# 파생변수 생성

이해원

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [17]:
pd.set_option('display.max_columns', None)

In [58]:
cont = pd.read_csv('SDB2_I_CT_CONT_PVT.csv')
rlps = pd.read_csv('rlps_fin3.csv')

In [59]:
rlps.head()

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,rltn_cd_x
0,8,795034,I,1,0
1,8,3700186,I,1,0
2,8,6394418,I,99,2
3,8,6614027,I,1,0
4,8,7083655,I,1,0


CT_CNCLS_DT 보험계약체결월 NVARCHAR(6)

CT_TRMNT_DT 보험계약해지월 NVARCHAR(6)

CT_ST_DT 보험시기월 NVARCHAR(6)

CT_END_DT 보험종기월 NVARCHAR(6)

CT_EFCTV_DT 보험실효월 NVARCHAR(6)

CT_RST_DT 보험부활월 NVARCHAR(6)

CT_PY_AMT 납입보험료 BIGINT

CT_PY_CYCLE_CD 납입주기코드 NVARCHAR(2)

CT_PY_PD 납입기간 BIGINT

# 0. 전처리

- 드랍 리스트: 업권코드, 인수심사결과코드, 진단계약여부, 계약체결/해지??/시기/종기/실효/부활월, 연월컬름들(20개+_정도)

## (1) 이상치 제거

In [60]:
pre_0 = cont.groupby('POL_SN')[['YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006']].sum().reset_index()

#이상치만 모으기
pre = pre_0[pre_0['YM_202006']>1]#13882개의 증권번호

#이상치 제거
cont = cont[~cont['POL_SN'].isin(pre['POL_SN'])].reset_index(drop=True)
display(cont)

,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,0,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,0,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,0,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,0,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,0,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303237,8,2330274,1,4,5,201905,999912,201905,202005,999912,999912,59,3,1,0,1,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
303238,8,6242384,0,7,2,201212,999912,201212,203812,999912,999912,47,2,10,0,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303239,6,6050915,0,7,7,200902,201107,200902,201902,999912,999912,15,2,10,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303240,6,9901184,0,14,2,201809,999912,201809,201902,999912,999912,96,1,0,0,1,5,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
cont.columns

Index(['SCTR_CD', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD', 'INS_CT_STAT_CD',
       'CT_CNCLS_DT', 'CT_TRMNT_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_EFCTV_DT',
       'CT_RST_DT', 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD', 'IS_CT_DIGNS',
       'CT_JDGMT_CD', 'CT_CHNL_CD', 'YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006'],
      dtype='object')

## (2) 단체보험 drop

In [62]:
cont = cont.loc[cont['IS_GRP_INS']==0]
cont.drop(columns =['IS_GRP_INS'], inplace=True)
cont.head()

,SCTR_CD,POL_SN,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## (3) 데이터프레임 정리

In [69]:
rlps.head()

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,rltn_cd_x
0,8,795034,I,1,0
1,8,3700186,I,1,0
2,8,6394418,I,99,2
3,8,6614027,I,1,0
4,8,7083655,I,1,0


In [71]:
rlps = rlps[['JOIN_SN', 'POL_SN']]
rlps

,JOIN_SN,POL_SN
0,8,795034
1,8,3700186
2,8,6394418
3,8,6614027
4,8,7083655
...,...,...
196190,1959698,7255079
196191,1959698,7435599
196192,1959698,8764404
196193,1959919,2748732


In [72]:
cont

,SCTR_CD,POL_SN,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,6,7995898,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,8,9743978,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,6,9040681,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303236,8,8914008,11,2,201901,999912,201901,202901,999912,999912,360,2,5,0,1,4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303238,8,6242384,7,2,201212,999912,201212,203812,999912,999912,47,2,10,0,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303239,6,6050915,7,7,200902,201107,200902,201902,999912,999912,15,2,10,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
303240,6,9901184,14,2,201809,999912,201809,201902,999912,999912,96,1,0,0,1,5,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
# 증권번호를 기준으로 cont에 left join
merged = pd.merge(cont, rlps, on='POL_SN', how='left')
merged.head()

,SCTR_CD,POL_SN,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006,JOIN_SN
0,6,3023157,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,323090.0
1,8,2067576,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,354843.0
2,6,7995898,7,2,201409,999912,201409,999912,999912,999912,52,2,30,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,249476.0
3,8,9743978,4,7,201604,201704,201604,201704,999912,999912,170,3,1,0,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,479886.0
4,6,9040681,4,2,201804,999912,201804,204804,999912,202006,43,2,25,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1077062.0


In [97]:
merged = merged[['JOIN_SN', 'POL_SN', 'SCTR_CD',
                 'INS_CT_STAT_CD', 'CT_CNCLS_DT', 'CT_TRMNT_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_EFCTV_DT', 'CT_RST_DT',
                 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD']]
merged.head()

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
0,323090.0,3023157,6,7,200806,200903,200806,200906,999912,999912,10,2,1
1,354843.0,2067576,8,2,201012,999912,201012,999912,999912,999912,33,2,20
2,249476.0,7995898,6,2,201409,999912,201409,999912,999912,999912,52,2,30
3,479886.0,9743978,8,7,201604,201704,201604,201704,999912,999912,170,3,1
4,1077062.0,9040681,6,2,201804,999912,201804,204804,999912,202006,43,2,25


In [77]:
# merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300388 entries, 0 to 300387
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   JOIN_SN         228710 non-null  float64
 1   POL_SN          300388 non-null  int64  
 2   SCTR_CD         300388 non-null  int64  
 3   INS_CT_STAT_CD  300388 non-null  int64  
 4   CT_CNCLS_DT     300388 non-null  int64  
 5   CT_TRMNT_DT     300388 non-null  int64  
 6   CT_ST_DT        300388 non-null  int64  
 7   CT_END_DT       300388 non-null  int64  
 8   CT_EFCTV_DT     300388 non-null  int64  
 9   CT_RST_DT       300388 non-null  int64  
 10  CT_PY_AMT       300388 non-null  int64  
 11  CT_PY_CYCLE_CD  300388 non-null  int64  
 12  CT_PY_PD        300388 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 29.8 MB


In [98]:
# JOIN_SN NULL 삭제
merged = merged.dropna(subset=['JOIN_SN'])

# JOIN_SN int64로 변경
merged['JOIN_SN'] = merged['JOIN_SN'].astype('int64')
merged

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
0,323090,3023157,6,7,200806,200903,200806,200906,999912,999912,10,2,1
1,354843,2067576,8,2,201012,999912,201012,999912,999912,999912,33,2,20
2,249476,7995898,6,2,201409,999912,201409,999912,999912,999912,52,2,30
3,479886,9743978,8,7,201604,201704,201604,201704,999912,999912,170,3,1
4,1077062,9040681,6,2,201804,999912,201804,204804,999912,202006,43,2,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300382,1278572,4863528,6,2,201711,999912,201711,999912,999912,999912,67,2,30
300383,1465232,8914008,8,2,201901,999912,201901,202901,999912,999912,360,2,5
300385,1692899,6050915,6,7,200902,201107,200902,201902,999912,999912,15,2,10
300386,290390,9901184,6,2,201809,999912,201809,201902,999912,999912,96,1,0


In [101]:
merged_sort = merged.sort_values(by='JOIN_SN')
merged_sort

,JOIN_SN,POL_SN,SCTR_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD
256112,8,8841475,6,2,200908,999912,200908,208908,999912,999912,91,2,15
250978,8,3700186,3,2,201110,999912,201110,206910,999912,999912,30,2,15
193156,8,8841475,6,2,200908,999912,200908,208908,999912,999912,88,2,15
38656,8,795034,3,2,201601,999912,201601,209001,999912,999912,94,2,20
188293,8,7083655,3,2,201601,999912,201601,202601,999912,999912,21,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200206,1959698,6321965,6,2,200712,999912,200712,206512,999912,999912,100,2,15
187139,1959698,7255079,8,2,201407,999912,201407,204007,999912,999912,58,2,20
28777,1959698,4808992,6,2,201402,999912,201402,203402,999912,999912,13,2,20
172042,1959919,3831658,3,1,201904,999912,201904,999912,999912,999912,80,2,5


- 개인에 대한 전체계약건수 대비 해지건수 

- 증권번호로 개인에 대한 데이터는 다 있으니 / 증권번호의 갯수를 카운트 후 / 해지건수만 카운트하여 비율 구하기

# 1. 개인별 보험계약건수

In [104]:
join_sn_counts = merged_sort['JOIN_SN'].value_counts().reset_index()
join_sn_counts.columns = ['JOIN_SN', '보험계약건수']
join_sn_counts

,JOIN_SN,보험계약건수
0,357819,60
1,1132460,56
2,1878719,47
3,39352,46
4,1496597,46
...,...,...
39953,1289372,1
39954,1749569,1
39955,857020,1
39956,857158,1


# 2. 해지건수

- 개인에 대한 전체계약건수 대비 해지건수 

- 증권번호로 개인에 대한 데이터는 다 있으니 / 증권번호의 갯수를 카운트 후 / 해지건수만 카운트하여 비율 구하기

In [105]:
ins_cd_7_counts = merged_sort[merged_sort['INS_CT_STAT_CD'] == 7]['JOIN_SN'].value_counts().reset_index()
ins_cd_7_counts.columns = ['JOIN_SN', '해지건수']
ins_cd_7_counts

,JOIN_SN,해지건수
0,748477,25
1,19979,23
2,1226045,23
3,1433558,20
4,1166691,20
...,...,...
19921,593725,1
19922,1259708,1
19923,1259764,1
19924,593372,1


# 3. 만기건수

In [106]:
ins_cd_5_counts = merged_sort[merged_sort['INS_CT_STAT_CD'] == 5]['JOIN_SN'].value_counts().reset_index()
ins_cd_5_counts.columns = ['JOIN_SN', '만기건수']
ins_cd_5_counts

,JOIN_SN,만기건수
0,1026251,26
1,1750242,23
2,1910039,23
3,195798,22
4,833732,22
...,...,...
11075,1105971,1
11076,430960,1
11077,431125,1
11078,431181,1


# 4. 데이터프레임 합치기

In [108]:
new = pd.merge(join_sn_counts, ins_cd_7_counts, on='JOIN_SN', how='left')
new = pd.merge(new, ins_cd_5_counts, on='JOIN_SN', how='left')
new = new.fillna(0)
new

,JOIN_SN,보험계약건수,해지건수,만기건수
0,357819,60,1.0,17.0
1,1132460,56,2.0,0.0
2,1878719,47,0.0,0.0
3,39352,46,3.0,4.0
4,1496597,46,1.0,6.0
...,...,...,...,...
39953,1289372,1,0.0,0.0
39954,1749569,1,0.0,0.0
39955,857020,1,0.0,0.0
39956,857158,1,0.0,1.0


In [109]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39958 entries, 0 to 39957
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   JOIN_SN  39958 non-null  int64  
 1   보험계약건수   39958 non-null  int64  
 2   해지건수     39958 non-null  float64
 3   만기건수     39958 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 1.2 MB


In [112]:
new['해지건수'] = new['해지건수'].astype(int)
new['만기건수'] = new['만기건수'].astype(int)

new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39958 entries, 0 to 39957
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   JOIN_SN  39958 non-null  int64
 1   보험계약건수   39958 non-null  int64
 2   해지건수     39958 non-null  int32
 3   만기건수     39958 non-null  int32
dtypes: int32(2), int64(2)
memory usage: 936.6 KB


In [114]:
new['해지율'] = new['해지건수'] / new['보험계약건수'] 
new['만기율'] = new['만기건수'] / new['보험계약건수']
new

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
0,357819,60,1,17,0.016667,0.283333
1,1132460,56,2,0,0.035714,0.000000
2,1878719,47,0,0,0.000000,0.000000
3,39352,46,3,4,0.065217,0.086957
4,1496597,46,1,6,0.021739,0.130435
...,...,...,...,...,...,...
39953,1289372,1,0,0,0.000000,0.000000
39954,1749569,1,0,0,0.000000,0.000000
39955,857020,1,0,0,0.000000,0.000000
39956,857158,1,0,1,0.000000,1.000000


In [116]:
new['해지율'] = new['해지율'].round(2)
new['만기율'] = new['만기율'].round(2)
new

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
0,357819,60,1,17,0.02,0.28
1,1132460,56,2,0,0.04,0.00
2,1878719,47,0,0,0.00,0.00
3,39352,46,3,4,0.07,0.09
4,1496597,46,1,6,0.02,0.13
...,...,...,...,...,...,...
39953,1289372,1,0,0,0.00,0.00
39954,1749569,1,0,0,0.00,0.00
39955,857020,1,0,0,0.00,0.00
39956,857158,1,0,1,0.00,1.00


In [117]:
# new.to_csv('만기_해지_비율.csv', index=False)

In [118]:
ratio = pd.read_csv('만기_해지_비율.csv')
ratio

,JOIN_SN,보험계약건수,해지건수,만기건수,해지율,만기율
0,357819,60,1,17,0.02,0.28
1,1132460,56,2,0,0.04,0.00
2,1878719,47,0,0,0.00,0.00
3,39352,46,3,4,0.07,0.09
4,1496597,46,1,6,0.02,0.13
...,...,...,...,...,...,...
39953,1289372,1,0,0,0.00,0.00
39954,1749569,1,0,0,0.00,0.00
39955,857020,1,0,0,0.00,0.00
39956,857158,1,0,1,0.00,1.00
